# Losses, NAB scores and classification metrics analysis

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, roc_auc_score

In [3]:
res = pd.read_json(f"NAB/results/final_results.json")
scores = pd.read_csv(f"results/scores.csv")
losses_log_total = pd.read_csv(f"results/losses_log_total.csv")
losses_square_total = pd.read_csv(f"results/losses_square_total.csv")

## Total logarithmic and square losses

In [4]:
losses_log_total["folder_aggr"] = np.where(
    losses_log_total["folder_name"].str.contains("real"), "realTotal", "artTotal"
)
losses_log_group = losses_log_total.groupby(["folder_aggr"]).sum().T
losses_square_total["folder_aggr"] = np.where(
    losses_square_total["folder_name"].str.contains("real"), "realTotal", "artTotal"
)
losses_square_group = losses_square_total.groupby(["folder_aggr"]).sum().T
pd.concat(
    [losses_log_group.round().astype("int"), losses_square_group.round().astype("int")], axis=1
)

folder_aggr,artTotal,realTotal,artTotal,realTotal
loss_knncad,37548,280986,9528,84656
loss_numentaTM,10675,157598,2178,29573
loss_twitterADVec,84482,1072982,2446,31066
loss_skyline,55981,772310,2444,29684
loss_earthgeckoSkyline,83964,1079924,2431,31267
loss_numenta,11511,159686,2297,29602
loss_bayesChangePt,82850,1040745,2746,37193
loss_null,30742,222643,11088,80302
loss_expose,50065,457247,17407,150035
loss_relativeEntropy,86416,1075641,2502,31143


## Classification metrics

In [5]:
auc_score = pd.DataFrame(
    index=[
        "realAWSCloudwatch",
        "realTraffic",
        "realAdExchange",
        "realTweets",
        "artificialWithAnomaly",
        "realKnownCause",
        "Total",
    ]
)
alg_list = scores.filter(regex="^score", axis=1).columns.tolist()
alg_list = [i.replace("score_", "") for i in alg_list]
f1_score = auc_score.copy()
class_score = auc_score.copy()
thresholds_mat = auc_score.copy()
epsilon = 10 ** (-15)
for folder_name in auc_score.index:
    for alg_ind in alg_list:
        if folder_name == "Total":
            auc_score.loc["Total", alg_ind] = roc_auc_score(
                scores["label"], scores[f"score_{alg_ind}"]
            )
            precision, recall, thresholds = precision_recall_curve(
                scores["label"], scores[f"score_{alg_ind}"]
            )
            precision[(precision == 0) & (recall == 0)] = epsilon
            f1 = 2 * precision * recall / (precision + recall)
            f1_score.loc["Total", alg_ind] = np.max(f1)
            thresholds_mat.loc["Total", alg_ind] = (thresholds[f1[1:] == np.max(f1[1:])]).item()
            preds = np.where(
                scores[f"score_{alg_ind}"] >= (thresholds[f1[1:] == np.max(f1[1:])]).item(),
                1,
                0,
            )
            class_score.loc["Total", alg_ind] = sum(preds == scores["label"]) / scores.shape[0]
        else:
            scores_folder = scores[scores["folder_name"] == folder_name]
            auc_score.loc[folder_name, alg_ind] = roc_auc_score(
                scores_folder["label"], scores_folder[f"score_{alg_ind}"]
            )
            precision, recall, thresholds = precision_recall_curve(
                scores_folder["label"], scores_folder[f"score_{alg_ind}"]
            )
            precision[(precision == 0) & (recall == 0)] = epsilon
            f1 = 2 * precision * recall / (precision + recall)
            f1_score.loc[folder_name, alg_ind] = np.max(f1)
            thresholds_mat.loc[folder_name, alg_ind] = (
                thresholds[f1[1:] == np.max(f1[1:])]
            ).item()
            preds = np.where(
                scores_folder[f"score_{alg_ind}"]
                >= (thresholds[f1[1:] == np.max(f1[1:])]).item(),
                1,
                0,
            )
            class_score.loc[folder_name, alg_ind] = (
                sum(preds == scores_folder["label"]) / scores_folder.shape[0]
            )

### Area under curve

In [6]:
auc_score.T

,realAWSCloudwatch,realTraffic,realAdExchange,realTweets,artificialWithAnomaly,realKnownCause,Total
knncad,0.622818,0.589225,0.581913,0.618328,0.552503,0.522551,0.599931
numentaTM,0.522261,0.594351,0.450797,0.490430,0.544040,0.544955,0.498199
twitterADVec,0.502905,0.504703,0.504456,0.504055,0.503952,0.505891,0.504229
skyline,0.581308,0.517402,0.481112,0.548194,0.547815,0.640148,0.565846
earthgeckoSkyline,0.501455,0.503385,0.504167,0.500800,0.500322,0.500863,0.501144
numenta,0.532440,0.585984,0.530454,0.498449,0.524794,0.576805,0.515566
bayesChangePt,0.506174,0.505980,0.506904,0.500070,0.502676,0.502567,0.503310
null,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
expose,0.578061,0.537857,0.522085,0.542092,0.485839,0.598109,0.556560
relativeEntropy,0.504331,0.506132,0.504689,0.501486,0.503861,0.503004,0.502767


### F-score

In [7]:
f1_score.T

,realAWSCloudwatch,realTraffic,realAdExchange,realTweets,artificialWithAnomaly,realKnownCause,Total
knncad,0.239277,0.215415,0.235092,0.236698,0.218776,0.186825,0.215825
numentaTM,0.246931,0.286780,0.202619,0.179605,0.297935,0.268607,0.200521
twitterADVec,0.170475,0.181143,0.181646,0.179605,0.181736,0.173173,0.167872
skyline,0.216844,0.181143,0.181646,0.182153,0.245187,0.242292,0.190839
earthgeckoSkyline,0.170475,0.181143,0.181646,0.179605,0.181736,0.173173,0.167872
numenta,0.250269,0.275348,0.230937,0.179605,0.264591,0.282678,0.210155
bayesChangePt,0.170475,0.181143,0.181646,0.179605,0.181736,0.173173,0.167872
null,0.170475,0.181143,0.181646,0.179605,0.181736,0.173173,0.167872
expose,0.194479,0.187325,0.192412,0.183609,0.195908,0.223156,0.181409
relativeEntropy,0.170475,0.181143,0.181646,0.179605,0.181736,0.173173,0.167872


### Optimal threshold (maximising F-score)

In [8]:
thresholds_mat.T

,realAWSCloudwatch,realTraffic,realAdExchange,realTweets,artificialWithAnomaly,realKnownCause,Total
knncad,0.499145,0.415730,0.497797,0.499316,0.112821,0.499316,0.499316
numentaTM,0.032393,0.030103,0.039316,0.002258,0.133852,0.073306,0.030103
twitterADVec,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
skyline,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000
earthgeckoSkyline,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
numenta,0.030103,0.052550,0.038593,0.003042,0.042738,0.030103,0.030103
bayesChangePt,0.000000,0.001996,0.023256,0.017544,0.012739,0.000000,0.000000
null,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
expose,0.204153,0.870376,0.023034,0.760652,0.046420,0.807564,0.770708
relativeEntropy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Classification accuracy

In [9]:
class_score.T

,realAWSCloudwatch,realTraffic,realAdExchange,realTweets,artificialWithAnomaly,realKnownCause,Total
knncad,0.578122,0.492467,0.581374,0.519634,0.411913,0.501474,0.540267
numentaTM,0.856008,0.718910,0.878980,0.353292,0.881903,0.898751,0.863877
twitterADVec,0.093180,0.099591,0.099896,0.098663,0.099950,0.094794,0.091627
skyline,0.744080,0.757725,0.794901,0.098663,0.640005,0.602881,0.091627
earthgeckoSkyline,0.093180,0.099591,0.099896,0.098663,0.099950,0.094794,0.091627
numenta,0.855816,0.863317,0.779396,0.424545,0.843709,0.845675,0.866369
bayesChangePt,0.093180,0.872957,0.881790,0.849613,0.882771,0.094794,0.091627
null,0.093180,0.099591,0.099896,0.098663,0.099950,0.094794,0.091627
expose,0.450782,0.499745,0.474922,0.445235,0.319279,0.625250,0.556902
relativeEntropy,0.093180,0.099591,0.099896,0.098663,0.099950,0.094794,0.091627


## Total NAB scores and classification metrics

In [10]:
NAB_scores = pd.merge(
    res.T, pd.DataFrame(auc_score.loc["Total"]), left_index=True, right_index=True
).rename({"Total": "AUC"}, axis=1)

NAB_scores = pd.merge(
    NAB_scores, pd.DataFrame(f1_score.loc["Total"]), left_index=True, right_index=True
).rename({"Total": "F1-score"}, axis=1)

NAB_scores = pd.merge(
    NAB_scores, pd.DataFrame(class_score.loc["Total"]), left_index=True, right_index=True
).rename({"Total": "correct_classification_rate"}, axis=1)
NAB_scores.sort_values("standard", ascending=False).round(
    {
        "reward_low_FN_rate": 1,
        "reward_low_FP_rate": 1,
        "standard": 1,
        "AUC": 3,
        "F-score": 3,
        "correct_classification_rate": 3,
    }
)

,reward_low_FN_rate,reward_low_FP_rate,standard,AUC,F1-score,correct_classification_rate
Fixed10,98.8,97.1,98.2,0.998,0.990113,0.998
Fixed5,98.8,97.3,98.1,0.998,0.989431,0.998
Fixed20,98.2,96.1,97.4,0.998,0.988723,0.998
Variable50,97.7,93.6,96.6,0.998,0.988082,0.998
Variable70,97.7,93.3,96.5,0.998,0.987941,0.998
Variable10,97.6,93.3,96.4,0.998,0.984331,0.997
Variable20,97.6,93.3,96.4,0.998,0.985868,0.997
Variable30,97.6,93.2,96.4,0.998,0.986956,0.998
Fixed30,97.3,95.1,96.2,0.998,0.984181,0.997
Variable5,97.4,93.3,96.1,0.998,0.982680,0.997
